# Overview
- kaggle_nb001ベースで、分類クラスではなく確率を出力するように変更。

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from common import gini_coefficient

pd.set_option('display.max_columns', 100)


## 1) データ読み込み

In [2]:
# trainデータ
# -----------
data_train = pd.read_csv('../data/train_nb003.csv', index_col=0)
data_train.tail()

,ps_ind_01,ps_ind_03,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin,ps_ind_02_cat_2,ps_ind_02_cat_3,ps_ind_02_cat_4,ps_ind_04_cat_1,ps_ind_05_cat_1,ps_ind_05_cat_2,ps_ind_05_cat_3,ps_ind_05_cat_4,ps_ind_05_cat_6,ps_car_01_cat_4,ps_car_01_cat_5,...,ps_reg_01 ps_car_15,ps_reg_01 ps_calc_01,ps_reg_01 ps_calc_02,ps_reg_01 ps_calc_03,ps_reg_02^2,ps_reg_02 ps_reg_03,ps_reg_02 ps_car_12,ps_reg_02 ps_car_13,ps_reg_02 ps_car_14,ps_reg_02 ps_car_15,ps_reg_02 ps_calc_01,ps_reg_02 ps_calc_02,ps_reg_02 ps_calc_03,ps_reg_03^2,ps_reg_03 ps_car_12,ps_reg_03 ps_car_13,ps_reg_03 ps_car_14,ps_reg_03 ps_car_15,ps_reg_03 ps_calc_01,ps_reg_03 ps_calc_02,ps_reg_03 ps_calc_03,ps_car_12^2,ps_car_12 ps_car_13,ps_car_12 ps_car_14,ps_car_12 ps_car_15,ps_car_12 ps_calc_01,ps_car_12 ps_calc_02,ps_car_12 ps_calc_03,ps_car_13^2,ps_car_13 ps_car_14,ps_car_13 ps_car_15,ps_car_13 ps_calc_01,ps_car_13 ps_calc_02,ps_car_13 ps_calc_03,ps_car_14^2,ps_car_14 ps_car_15,ps_car_14 ps_calc_01,ps_car_14 ps_calc_02,ps_car_14 ps_calc_03,ps_car_15^2,ps_car_15 ps_calc_01,ps_car_15 ps_calc_02,ps_car_15 ps_calc_03,ps_calc_01^2,ps_calc_01 ps_calc_02,ps_calc_01 ps_calc_03,ps_calc_02^2,ps_calc_02 ps_calc_03,ps_calc_03^2,target
216935,5,7,0,0,1,0,0,8,0,1,0,0.6,0.4,1.537652,1,0.424264,1.269111,0.384708,3.162278,0.5,0.1,0.5,3,4,9,1,10,2,9,10,1,5,7,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,1.897367,0.30,0.06,0.30,0.16,0.615061,0.169706,0.507645,0.153883,1.264911,0.20,0.04,0.20,2.364375,0.652371,1.951452,0.591547,4.862484,0.768826,0.153765,0.768826,0.18,0.538438,0.163218,1.341641,0.212132,0.042426,0.212132,1.610644,0.488237,4.013282,0.634556,0.126911,0.634556,0.1480,1.216553,0.192354,0.038471,0.192354,10.0,1.581139,0.316228,1.581139,0.25,0.05,0.25,0.01,0.05,0.25,1
216936,1,5,0,0,1,0,0,4,0,0,1,0.3,0.4,0.897653,2,0.424264,0.756979,0.400000,2.000000,0.3,0.4,0.6,1,1,8,2,9,5,9,9,2,1,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.600000,0.09,0.12,0.18,0.16,0.359061,0.169706,0.302791,0.160000,0.800000,0.12,0.16,0.24,0.805782,0.380842,0.679504,0.359061,1.795307,0.269296,0.359061,0.538592,0.18,0.321159,0.169706,0.848528,0.127279,0.169706,0.254558,0.573017,0.302791,1.513957,0.227094,0.302791,0.454187,0.1600,0.800000,0.120000,0.160000,0.240000,4.0,0.600000,0.800000,1.200000,0.09,0.12,0.18,0.16,0.24,0.36,1
216937,0,6,0,1,0,0,0,9,0,1,0,0.9,0.3,0.711952,3,0.400000,0.970654,0.372424,3.464102,0.5,0.4,0.6,2,2,8,2,8,4,10,3,2,1,6,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,3.117691,0.45,0.36,0.54,0.09,0.213585,0.120000,0.291196,0.111727,1.039230,0.15,0.12,0.18,0.506875,0.284781,0.691059,0.265148,2.466272,0.355976,0.284781,0.427171,0.16,0.388262,0.148970,1.385641,0.200000,0.160000,0.240000,0.942169,0.361495,3.362445,0.485327,0.388262,0.582392,0.1387,1.290116,0.186212,0.148970,0.223455,12.0,1.732051,1.385641,2.078461,0.25,0.20,0.30,0.16,0.24,0.36,1
216938,0,2,1,0,0,0,0,2,0,0,1,0.6,0.1,0.577170,2,0.316228,0.876295,0.320780,3.741657,0.5,0.2,0.6,3,0,9,3,10,4,11,5,0,4,4,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,...,2.244994,0.30,0.12,0.36,0.01,0.057717,0.031623,0.087629,0.032078,0.374166,0.05,0.02,0.06,0.333125,0.182517,0.505771,0.185145,2.159572,0.288585,0.115434,0.346302,0.10,0.277109,0.101440,1.183216,0.158114,0.063246,0.189737,0.767893,0.281098,3.278795,0.438147,0.175259,0.525777,0.1029,1.200250,0.160390,0.064156,0.192468,14.0,1.870829,0.748331,2.244994,0.25,0.10,0.30,0.04,0.12,0.36,1
216939,1,1,0,0,1,0,0,13,1,0,0,0.6,0.4,1.095160,2,0.374166,0.752558,0.328634,3.464102,0.2,0.6,0.3,1,1,8,2,11,0,13,7,2,4,9,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,...,2.078461,0.12,0.36,0.18,0.16,0.438064,0.149666,0.301023,0.131453,1.385641,0.08,0.24,0.12,1.199375,0.4

In [3]:
# testデータ
# ----------
data_test = pd.read_csv('../data/test_nb003.csv', index_col=0)
data_test.tail()

,ps_ind_01,ps_ind_03,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_14,ps_ind_15,ps_ind_16_bin,ps_ind_17_bin,ps_ind_18_bin,ps_reg_01,ps_reg_02,ps_reg_03,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15,ps_calc_01,ps_calc_02,ps_calc_03,ps_calc_04,ps_calc_05,ps_calc_06,ps_calc_07,ps_calc_08,ps_calc_09,ps_calc_10,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin,ps_ind_02_cat_2,ps_ind_02_cat_3,ps_ind_02_cat_4,ps_ind_04_cat_1,ps_ind_05_cat_1,ps_ind_05_cat_2,ps_ind_05_cat_3,ps_ind_05_cat_4,ps_ind_05_cat_6,ps_car_01_cat_4,ps_car_01_cat_5,...,ps_reg_01 ps_car_14,ps_reg_01 ps_car_15,ps_reg_01 ps_calc_01,ps_reg_01 ps_calc_02,ps_reg_01 ps_calc_03,ps_reg_02^2,ps_reg_02 ps_reg_03,ps_reg_02 ps_car_12,ps_reg_02 ps_car_13,ps_reg_02 ps_car_14,ps_reg_02 ps_car_15,ps_reg_02 ps_calc_01,ps_reg_02 ps_calc_02,ps_reg_02 ps_calc_03,ps_reg_03^2,ps_reg_03 ps_car_12,ps_reg_03 ps_car_13,ps_reg_03 ps_car_14,ps_reg_03 ps_car_15,ps_reg_03 ps_calc_01,ps_reg_03 ps_calc_02,ps_reg_03 ps_calc_03,ps_car_12^2,ps_car_12 ps_car_13,ps_car_12 ps_car_14,ps_car_12 ps_car_15,ps_car_12 ps_calc_01,ps_car_12 ps_calc_02,ps_car_12 ps_calc_03,ps_car_13^2,ps_car_13 ps_car_14,ps_car_13 ps_car_15,ps_car_13 ps_calc_01,ps_car_13 ps_calc_02,ps_car_13 ps_calc_03,ps_car_14^2,ps_car_14 ps_car_15,ps_car_14 ps_calc_01,ps_car_14 ps_calc_02,ps_car_14 ps_calc_03,ps_car_15^2,ps_car_15 ps_calc_01,ps_car_15 ps_calc_02,ps_car_15 ps_calc_03,ps_calc_01^2,ps_calc_01 ps_calc_02,ps_calc_01 ps_calc_03,ps_calc_02^2,ps_calc_02 ps_calc_03,ps_calc_03^2
892811,0,6,0,1,0,0,0,2,0,0,1,0.5,0.3,1.048809,2,0.565685,1.528774,0.565685,3.605551,0.3,0.4,0.9,1,1,7,3,10,3,10,4,2,3,4,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0.282843,1.802776,0.15,0.20,0.45,0.09,0.314643,0.169706,0.458632,0.169706,1.081665,0.09,0.12,0.27,1.100000,0.593296,1.603391,0.593296,3.781534,0.314643,0.419524,0.943928,0.32,0.864805,0.320000,2.039608,0.169706,0.226274,0.509117,2.337149,0.864805,5.512072,0.458632,0.611509,1.375896,0.3200,2.039608,0.169706,0.226274,0.509117,13.0,1.081665,1.442221,3.244996,0.09,0.12,0.27,0.16,0.36,0.81
892812,5,5,0,0,1,0,0,11,1,0,0,0.7,1.0,1.246495,2,0.424264,1.040567,0.418569,3.741657,0.3,0.2,0.6,1,3,7,3,10,4,9,6,2,2,11,0,0,1,1,0,0,0,1,0,1,0,0,0,0,0,0,0,...,0.292998,2.619160,0.21,0.14,0.42,1.00,1.246495,0.424264,1.040567,0.418569,3.741657,0.30,0.20,0.60,1.553750,0.528843,1.297062,0.521744,4.663958,0.373949,0.249299,0.747897,0.18,0.441475,0.177584,1.587451,0.127279,0.084853,0.254558,1.082780,0.435549,3.893446,0.312170,0.208113,0.624340,0.1752,1.566142,0.125571,0.083714,0.251141,14.0,1.122497,0.748331,2.244994,0.09,0.06,0.18,0.04,0.12,0.36
892813,0,5,1,0,0,0,0,5,0,0,1,0.4,0.0,0.609303,2,0.374166,0.968992,0.390384,3.741657,0.3,0.3,0.9,2,1,10,4,8,2,8,5,2,2,11,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.156154,1.496663,0.12,0.12,0.36,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.371250,0.227980,0.590410,0.237862,2.279803,0.182791,0.182791,0.548373,0.14,0.362564,0.146068,1.400000,0.112250,0.112250,0.336749,0.938946,0.378280,3.625638,0.290698,0.290698,0.872093,0.1524,1.460685,0.117115,0.117115,0.351346,14.0,1.122497,1.122497,3.367492,0.09,0.09,0.27,0.09,0.27,0.81
892814,6,5,0,0,0,1,0,13,1,0,0,0.6,0.6,0.920937,3,0.387298,0.725125,0.378153,2.645751,0.1,0.1,0.3,1,1,9,2,10,1,8,1,1,2,7,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0.226892,1.587451,0.06,0.06,0.18,0.36,0.552562,0.232379,0.435075,0.226892,1.587451,0.06,0.06,0.18,0.848125,0.356677,0.667794,0.348255,2.436570,0.092094,0.092094,0.276281,0.15,0.280840,0.146458,1.024695,0.038730,0.038730,0.116190,0.525806,0.274208,1.918500,0.072512,0.072512,0.217537,0.1430,1.000500,0.037815,0.037815,0.113446,7.0,0.264575,0.264575,0.793725,0.01,0.01,0.03,0.01,0.03,0.09
892815,7,4,0,0,0,1,0,12,1,0,0,0.9,0.8,0.992157,1,0.316228,0.637175,0.296648,3.316625,0.4,0.4,0.2,3,4,7,2,8,4,15,5,2,2,7,0,1,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0.266983,2.984962,0.36,0.36,0.18,0.64,0.793725,0.252982,0

In [4]:
# testデータ(id)
# --------------
id_test = pd.read_csv('../data/id_test_nb003.csv', index_col=0)
id_test.tail()

,id
892811,1488022
892812,1488023
892813,1488024
892814,1488025
892815,1488026


In [5]:
X = data_train.drop('target', axis=1)
y = data_train['target']
X_test = data_test

## 2) 解析

In [6]:
# 訓練用、チェック用にデータ分割
# -------------------------------
X_train, X_check, y_train, y_check = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)   # 訓練:テスト = 80:20

print('Label counts in y_train: [0 1] =', np.bincount(y_train.astype(np.int64)))
print('Label counts in y_check: [0 1] =', np.bincount(y_check.astype(np.int64)))

Label counts in y_train: [0 1] = [156197  17355]
Label counts in y_check: [0 1] = [39049  4339]


In [6]:
# =================================
# Pipeline: pl_svc
# SVC / k分割交差検証 / グリッドサーチ
# =================================

from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV

# SVCパイプライン作成
pl_svc = make_pipeline(StandardScaler(), SVC(random_state=21, max_iter=100, probability=True))

# 標準化gini係数を評価指標とする
normalized_gini = make_scorer(gini_coefficient.gini_norm, greater_is_better=True)
# 評価指標は引数に(y_true(正解データ), y_pred(予測データ))を呼ぶ必要がある

### Grid Search試行用 ###
#svc_param_range = [1.0]
#svc_param_grid = [{'svc__C': svc_param_range, 'svc__kernel': ['rbf'], 'svc__gamma': svc_param_range}]

### Grid Search本番用 1 ###
svc_param_range = [0.01, 0.1, 1.0, 10.0]
svc_param_grid = [{'svc__C': svc_param_range, 'svc__kernel': ['rbf'], 'svc__gamma': svc_param_range}]

### Grid Search本番用 2 ###
#svc_param_range = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0]
#svc_param_grid = [{'svc__C': svc_param_range, 'svc__kernel': ['linear']},
#                  {'svc__C': svc_param_range, 'svc__kernel': ['poly', 'rbf', 'sigmoid'], 'svc__gamma': svc_param_range}]


svc_gs = GridSearchCV(estimator=pl_svc, param_grid=svc_param_grid, scoring=normalized_gini, cv=5, refit=True, n_jobs=-1)
svc_gs.fit(X_train, y_train)

print('Grid Search best score:', svc_gs.best_score_)
print('Best parameters:', svc_gs.best_params_)
svc_bestclf = svc_gs.best_estimator_


Grid Search best score: -0.6806481092002146
Best parameters: {'svc__C': 10.0, 'svc__gamma': 0.01, 'svc__kernel': 'rbf'}


C:\Users\ultra\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:255: ConvergenceWarning: Solver terminated early (max_iter=100).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn('Solver terminated early (max_iter=%i).'


In [7]:
pd.DataFrame(svc_gs.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_svc__C,param_svc__gamma,param_svc__kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,122.987390,0.517446,3.738557,0.241389,0.01,0.01,rbf,"{'svc__C': 0.01, 'svc__gamma': 0.01, 'svc__ker...",-0.859312,-0.833914,-0.798496,-0.846130,-0.865737,-0.840718,0.023791,8
1,120.591371,2.343654,3.438605,0.028810,0.01,0.1,rbf,"{'svc__C': 0.01, 'svc__gamma': 0.1, 'svc__kern...",-0.959427,-0.986760,-0.949331,-0.981944,-0.915427,-0.958578,0.025645,9
2,112.092703,5.034203,3.420405,0.094074,0.01,1.0,rbf,"{'svc__C': 0.01, 'svc__gamma': 1.0, 'svc__kern...",-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000,16
3,119.031251,5.129883,4.483797,0.271487,0.01,10.0,rbf,"{'svc__C': 0.01, 'svc__gamma': 10.0, 'svc__ker...",-0.830066,-0.824194,-0.829825,-0.826169,-0.823376,-0.826726,0.002782,6
4,112.515265,7.852718,3.880239,0.649569,0.1,0.01,rbf,"{'svc__C': 0.1, 'svc__gamma': 0.01, 'svc__kern...",-0.807610,-0.853510,-0.913735,-0.702329,-0.865578,-0.828552,0.071583,7
5,131.582903,3.047996,4.144931,0.588631,0.1,0.1,rbf,"{'svc__C': 0.1, 'svc__gamma': 0.1, 'svc__kerne...",-0.996316,-0.996781,-0.866567,-0.999078,-0.998161,-0.971381,0.052416,10
6,116.997390,12.421221,3.573705,0.361205,0.1,1.0,rbf,"{'svc__C': 0.1, 'svc__gamma': 1.0, 'svc__kerne...",-1.000000,-1.000000,-1.000000,-1.000000,-0.999539,-0.999908,0.000184,14
7,118.401367,3.995590,4.797905,0.156311,0.1,10.0,rbf,"{'svc__C': 0.1, 'svc__gamma': 10.0, 'svc__kern...",-0.827583,-0.821282,-0.826771,-0.822875,-0.821090,-0.823920,0.002742,5
8,107.188138,2.018540,3.591948,0.475764,1.0,0.01,rbf,"{'svc__C': 1.0, 'svc__gamma': 0.01, 'svc__kern...",-0.795507,-0.813478,-0.788981,-0.861924,-0.816737,-0.815326,0.025549,2
9,109.959395,0.881019,3.626808,0.380634,1.0,0.1,rbf,"{'svc__C': 1.0, 'svc__gamma': 0.1, 'svc__kerne...",-0.998157,-0.998158,-0.999079,-0.998619,-0.999540,-0.998711,0.000537,13


In [8]:
# testデータで予測
# =================

#svc_pred = svc_bestclf.predict(X_test)
#submission = pd.DataFrame({"id": id_test['id'], "target": svc_pred})

svc_pred_proba = svc_bestclf.predict_proba(X_test)
submission_proba = pd.DataFrame({"id": id_test['id'], "target": svc_pred_proba[:,1]})

# csvに出力
#submission_proba.to_csv("submission_nb005_svc.csv", index = False)

submission_proba

,id,target
0,0,0.104394
1,1,0.092384
2,2,0.111295
3,3,0.101799
4,4,0.084559
...,...,...
892811,1488022,0.086486
892812,1488023,0.089470
892813,1488024,0.104811
892814,1488025,0.087496


In [9]:
max(submission_proba['target'])

0.17570450052268063

In [10]:
# =============================================
# Pipeline: pl_randf
# ランダムフォレスト / k分割交差検証 / グリッドサーチ
# =============================================

from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV

# Random Forestパイプライン作成
pl_randf = make_pipeline(RandomForestClassifier(random_state=21, n_jobs=-1))

# 標準化gini係数を評価指標とする
normalized_gini = make_scorer(gini_coefficient.gini_norm, greater_is_better=True)
# 評価指標は引数に(y_true(正解データ), y_pred(予測データ))を呼ぶ必要がある

from sklearn.model_selection import GridSearchCV

### Grid Search試行用 ###
rf_param_estimators_range = [100, 200]
rf_param_depth_range = [5, 10]
rf_param_split_range = [5, 10]

### Grid Search本番用 ###
#rf_param_estimators_range = [100, 200, 300, 400, 500]
#rf_param_depth_range = [5, 10, 15, 20, 25, 30]
#rf_param_split_range = [5, 10, 15, 20, 25, 30]

rf_param_grid = [{'randomforestclassifier__criterion': ['gini', 'entropy'],
                  'randomforestclassifier__n_estimators': rf_param_estimators_range,
                  'randomforestclassifier__max_depth': rf_param_depth_range,
                  'randomforestclassifier__min_samples_split': rf_param_split_range}]
rf_gs = GridSearchCV(estimator=pl_randf, param_grid=rf_param_grid, scoring=normalized_gini, cv=5, refit=True, n_jobs=-1)
rf_gs.fit(X_train, y_train)

print('Grid Search best score:', rf_gs.best_score_)
print(rf_gs.best_params_)
rf_bestclf = rf_gs.best_estimator_


Grid Search best score: -1.0
{'randomforestclassifier__criterion': 'gini', 'randomforestclassifier__max_depth': 5, 'randomforestclassifier__min_samples_split': 5, 'randomforestclassifier__n_estimators': 100}


In [11]:
pd.DataFrame(rf_gs.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_randomforestclassifier__criterion,param_randomforestclassifier__max_depth,param_randomforestclassifier__min_samples_split,param_randomforestclassifier__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,71.352994,0.494250,1.031113,0.890171,gini,5,5,100,"{'randomforestclassifier__criterion': 'gini', ...",-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1
1,142.811194,2.013105,2.372721,0.755152,gini,5,5,200,"{'randomforestclassifier__criterion': 'gini', ...",-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1
2,77.026640,4.072920,1.466053,0.170505,gini,5,10,100,"{'randomforestclassifier__criterion': 'gini', ...",-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1
3,150.450235,2.386379,2.350496,0.579362,gini,5,10,200,"{'randomforestclassifier__criterion': 'gini', ...",-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1
4,142.759774,1.278026,1.946390,0.808856,gini,10,5,100,"{'randomforestclassifier__criterion': 'gini', ...",-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1
5,276.765454,2.251870,3.676490,0.392751,gini,10,5,200,"{'randomforestclassifier__criterion': 'gini', ...",-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1
6,136.584903,2.163353,2.315699,0.597967,gini,10,10,100,"{'randomforestclassifier__criterion': 'gini', ...",-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1
7,266.997565,6.969646,3.835056,0.608970,gini,10,10,200,"{'randomforestclassifier__criterion': 'gini', ...",-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1
8,78.417261,1.295764,2.327884,0.357106,entropy,5,5,100,{'randomforestclassifier__criterion': 'entropy...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1
9,147.833899,2.101348,3.026304,0.340750,entropy,5,5,200,{'randomforestclassifier__criterion': 'entropy...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,1


In [10]:
# =============================================
# Model: mdl_xgb
# XGBoost / k分割交差検証 / グリッドサーチ
# =============================================

import xgboost as xgb

# データ加工
xgb_train = xgb.DMatrix(X_train.values, label=y_train)
xgb_check = xgb.DMatrix(X_check.values, label=y_check)
xgb_test = xgb.DMatrix(X_test.values)

# パラメータ設定
params = {'objective': 'reg:squarederror', 'silent':1, 'random_state':21}
num_round = 500
watchlist = [(xgb_train, 'train'), (xgb_check, 'check')]

# モデル構築
mdl_xgb = xgb.train(params, xgb_train, num_round, early_stopping_rounds=100,
                    evals=watchlist, feval=gini_coefficient.gini_xgb,
                    verbose_eval=100)

[17:23:13] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-rmse:0.40915	train-gini_xgb:0.22385	check-rmse:0.40957	check-gini_xgb:0.20413
[100]	train-rmse:0.27491	train-gini_xgb:0.55867	check-rmse:0.29933	check-gini_xgb:0.23832


In [17]:
# testデータで予測
# =================

xgb_pred = mdl_xgb.predict(xgb_test)
submission_proba = pd.DataFrame({"id": id_test['id'], "target": xgb_pred})

# csvに出力
#submission_proba.to_csv("submission_nb005_xgb.csv", index = False)

submission_proba

,id,target
0,0,0.085969
1,1,0.071848
2,2,0.067632
3,3,0.038519
4,4,0.092770
...,...,...
892811,1488022,0.072372
892812,1488023,0.121591
892813,1488024,0.066328
892814,1488025,0.058754
